This notebook downloads ERA20C precipitation data for the chosen year, using a web API. Since we want to have the total daily precipitation sum, we have to add day1 0:00-6:00 and day2 6:00-24:00 to have a full day (ECWMF starts day at 6:00)
Information is in de the text blocks and decriptions marked with #

Steps that are taken:
0. Check dates that are not uploaded yet
1. Download data 2 days 
2. Calculate total sum P by adding day1 0:00-6:00 and day2 6:00-24:00
3. Create new netcdf file
4. Convert to geotiff (float32)
5. Upload geotiff --> bucket
6. Upload bucket --> Google Earth Engine 
7. Add Meta data

In [ ]:
#Choose year(s) that you want to download 
yearStart = 2001
yearEnd = 2001

Log in too GCP and GEE

In [ ]:
!gcloud auth login #login google cloud
!pip install --upgrade earthengine-api
!earthengine authenticate #login in GEE

Look at files stored in Bucket on GCP

In [ ]:
#create list of data downloaded and converted (saved in bucket) 
bucket= 'gs://era-ml-upressing/ERA20C2000/'
lsturl = !gsutil ls $bucket
files=[]
for file in lsturl:
  files.append(file.split("/")[-1][0:8])
files=files[1:]
files


In [ ]:
#add day before year start so all days can be done, so for 2001--> 2000-12-31 since you need data of day before 0:00-6:00
#here a list of all dates that need to be downloaded are created
import calendar
dates=['20001231']
monthStart = 1
monthEnd = 12
for year in list(range(yearStart, yearEnd + 1)):
    for month in list(range(monthStart, monthEnd + 1)):
        numberOfDays = calendar.monthrange(year, month)[1]
        for day in list(range(1,numberOfDays+1)):
          date = '%04d%02d%02d' % (year, month, day)
          dates.append(date)          
dates

In [ ]:
!pip install ecmwf-api-client
!pip install --user ecmwf-api-client
!apt install file netcdf-bin #to create netcdf files
!pip install xarray==0.15.1
# download data from ECMWF
import calendar
import numpy as np
import xarray
from ecmwfapi import ECMWFDataServer
server = ECMWFDataServer(url="https://api.ecmwf.int/v1",key=**,email=**)
for date in dates:
  target = date + ".nc"
  server.retrieve({
      'dataset' : 'era20c',
      'stream'  : 'oper',
      'levtype' : 'sfc',
      'param'   : '228.128',
      'date'    : date,
      'type'    : 'fc',
      'time'    : '06',
      'step'    : '18/24',
      'area'    : "90/-180/-90/180",
      'grid'    : '1/1',
      'target'  : target,
      'format'  : 'netcdf'
      })
for numdate in range(len(dates)-1):
  date=dates[numdate]
  filename = date + ".nc"
  nextday = dates[numdate + 1]
  year=nextday[:4]
  filenamenextday = nextday + ".nc"
  arr = xarray.open_dataset(filename)
  #get data 0-6AM nextday
  from_6_to_6 = arr.isel(time=1)
  from_6_to_24 =  arr.isel(time=0)
  from_24_to_6 = np.subtract(from_6_to_6.tp, from_6_to_24.tp)
  #add 6-24h data nextday to have data between 0-24h
  arr2 = xarray.open_dataset(filenamenextday)
  tempfile = np.add(arr2.isel(time=0).tp, from_24_to_6)
  tempfile.values[tempfile.values < 0] = 0
  newfilename = nextday + "data.nc"
  tempfile.to_netcdf(newfilename)     #save added data as nc file
  nametiff = nextday + '.tiff'
  !gdal_translate -a_srs WGS84 -unscale -ot Float32 NETCDF:$newfilename $nametiff  #convert from netcdf to geotiff, name= data.tiff
  !gsutil cp $nametiff $bucket #copy dataset to bucket
  image_id = 'users/avelon98/ERA20C{year}/{nextday}'.format(nextday=nextday, year=year)
  URLBucket = bucket + nextday + '.tiff'
  !earthengine upload image --asset_id=$image_id $URLBucket #upload from bucket to GEE

In [ ]:
#add metadata for GEE: dates, units
for date in dates[1:]:
  year=date[:4]
  image_id = 'users/avelon98/ERA20C{year}/{date}'.format(date=date, year=year)
  image_collection_id = 'users/avelon98/ERA20C{year}'.format(year=year)
  date1=date[0:4]+'-'+date[4:6]+'-'+date[6:8]
  !earthengine asset set --time_start $date1 $image_id
for year in list(range(yearStart, yearEnd + 1)):
  image_collection_id = 'users/avelon98/ERA20C{year}'.format(year=year)
  !earthengine asset set -p 'unit=m' $image_collection_id
  !earthengine asset set -p 'variable=Total_precipition' $image_collection_id